### BERT Baseline

In [1]:
# !pip install sklearn
# !pip install ekphrasis
# !pip install transformers
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import transformers

from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import backend as K
from tensorflow import keras

import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
encoder = LabelEncoder()
encoder.classes_ = np.load('Data/classes.npy', allow_pickle=True)

In [5]:
tf.__version__

'2.6.0'

In [6]:
transformers.__version__

'4.16.2'

In [7]:
import sys

sys.path.append('./Preprocess')

from dataCollect import *

C:\Users\evanc\anaconda3\envs\tf-gpu\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\evanc\anaconda3\envs\tf-gpu\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [8]:
params = {'data_file' : 'Data/dataset.json', 'class_names' : 'Data/classes.npy'}

raw_data = get_annotated_data(params)

raw_data

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],normal
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],normal
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...",4,"[Caucasian, Women]",hatespeech,2,"[Women, Caucasian]",hatespeech,3,"[Women, Caucasian]",offensive,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",hatespeech
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...",217,"[Men, Women, Other]",offensive,199,[None],offensive,215,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",offensive
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",offensive
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],normal
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",hatespeech


In [9]:
params = {'data_file' : 'Data/dataset.json', 'class_names' : 'classes.npy', 'bert_tokens' : True, 'majority' : 2.0, 'include_special' : False, 'max_length' : 128.0, 'decay' : False, "type_attention": "softmax", "variance": 5.0, 'num_classes' : 3.0}
bert_embedding_data = collect_data(params)

Loading BERT tokenizer...
total_data 20148


100%|███████████████████████████████████| 20148/20148 [01:08<00:00, 293.04it/s]


attention_error: 0
no_majority: 919


In [10]:
bert_embedding_data.head()

,Post_id,Text,Attention,Label
0,1179055004553900032_twitter,"[101, 1045, 2123, 2102, 2228, 10047, 2893, 202...","[0.041666666666666664, 0.041666666666666664, 0...",normal
1,1179063826874032128_twitter,"[101, 2057, 3685, 3613, 4214, 9731, 10469, 201...","[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0...",normal
2,1178793830532956161_twitter,"[101, 6583, 26677, 8038, 3363, 9152, 13327, 20...","[0.09090909090909091, 0.09090909090909091, 0.0...",normal
3,1179088797964763136_twitter,"[101, 5310, 1045, 2572, 2978, 5457, 2522, 2480...","[0.0290389020798568, 0.0290389020798568, 0.029...",hatespeech
4,1179085312976445440_twitter,"[101, 2023, 7743, 1999, 2054, 7875, 12514, 212...","[0.029413453722818332, 0.029413453722818332, 0...",hatespeech


In [11]:
# generate token type ids as 0s

tti_list = []
for item in bert_embedding_data.Text:
    tti = [0] * len(item)
    tti_list.append(tti)
    
bert_embedding_data['Token_Type_Ids'] = tti_list

bert_embedding_data.head()

,Post_id,Text,Attention,Label,Token_Type_Ids
0,1179055004553900032_twitter,"[101, 1045, 2123, 2102, 2228, 10047, 2893, 202...","[0.041666666666666664, 0.041666666666666664, 0...",normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1179063826874032128_twitter,"[101, 2057, 3685, 3613, 4214, 9731, 10469, 201...","[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0...",normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1178793830532956161_twitter,"[101, 6583, 26677, 8038, 3363, 9152, 13327, 20...","[0.09090909090909091, 0.09090909090909091, 0.0...",normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,1179088797964763136_twitter,"[101, 5310, 1045, 2572, 2978, 5457, 2522, 2480...","[0.0290389020798568, 0.0290389020798568, 0.029...",hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1179085312976445440_twitter,"[101, 2023, 7743, 1999, 2054, 7875, 12514, 212...","[0.029413453722818332, 0.029413453722818332, 0...",hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
num = 5

bert_embedding_data.iloc[num]

print(len(bert_embedding_data.iloc[num]['Text']))
print(len(bert_embedding_data.iloc[num]['Attention']))
print(len(bert_embedding_data.iloc[num]['Token_Type_Ids']))

37
37
37


In [13]:
# combine Text, TTI, and Attention so we can test train split
bert_embedding_data['X'] = list(zip(bert_embedding_data['Text'], bert_embedding_data['Attention'], bert_embedding_data['Token_Type_Ids']))

#bert_embedding_data

X_input = bert_embedding_data['X'] 

# convert class label to 1 hot encoding
y_label = bert_embedding_data['Label']

y_label_numerical = y_label.apply(lambda x: 0 if x=='normal' else (1 if x=='offensive' else 2))

y_label_encoded = keras.utils.to_categorical(y_label_numerical, num_classes = 3, dtype = 'float32')

In [14]:
# test train split
X_train, X_test, y_train, y_test = train_test_split(X_input, y_label_encoded, test_size=0.2, stratify=y_label_encoded)

In [15]:
# check class distributions

count = [0, 0, 0]
for a in y_train:
    count = np.add(a, count)
    
count

count = [0, 0, 0]
for a in y_test:
    count = np.add(a, count)
    
count

array([1563., 1096., 1187.])

In [47]:
# separate X_Train and test convert to arrays

X_train_input_ids = np.asarray([np.asarray(x[0], dtype=object) for x in X_train], dtype=object)
X_test_input_ids = np.asarray([np.asarray(x[0], dtype=object) for x in X_test], dtype=object)                                

X_train_attention_mask = np.asarray([np.asarray(x[1], dtype=object) for x in X_train], dtype=object)
X_test_attention_mask = np.asarray([np.asarray(x[1], dtype=object) for x in X_test], dtype=object)    

X_train_token_type_ids = np.asarray([np.asarray(x[2], dtype=object) for x in X_train], dtype=object)
X_test_token_type_ids = np.asarray([np.asarray(x[2], dtype=object) for x in X_test], dtype=object)    

X_train_input_ids

array([array([101, 2020, 1996, 4889, 11397, 10823, 1998, 6696, 2015, 1998, 4004,
              2015, 2064, 2017, 2425, 2033, 2138, 1045, 2572, 3609, 6397, 102],
             dtype=object)                                                      ,
       array([101, 5310, 2467, 2038, 2245, 4013, 22776, 16614, 2021, 2002, 2788,
              2062, 3497, 2000, 7514, 2000, 1037, 11382, 3489, 2084, 2033, 5060,
              2033, 12411, 4502, 2072, 102], dtype=object)                      ,
       array([101, 15537, 2033, 1997, 1037, 2267, 28536, 2000, 2393, 1037, 11382,
              3489, 2155, 14591, 2037, 11060, 2160, 1996, 2269, 2001, 1037,
              23848, 2923, 12173, 2080, 2029, 2003, 2066, 1037, 2152, 2457, 3648,
              2061, 2027, 2020, 26108, 2084, 2087, 1997, 1996, 2060, 2493, 2945,
              102], dtype=object)                                                ,
       ...,
       array([101, 2023, 7743, 2003, 2061, 2128, 7559, 5732, 5223, 4613, 3475,
              1056

In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [40]:
def create_classification_model(hidden_size = 200, 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam()):
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])


    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics='accuracy')


    return classification_model

In [48]:
max_length = 80

try:
    del classification_model
except:
    pass

try:
    del bert_model
except:
    pass

tf.keras.backend.clear_session()
bert_model = TFBertModel.from_pretrained('bert-base-cased')

classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(0.00005))

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [49]:
classification_model.fit([X_train_input_ids, X_train_token_type_ids, X_train_attention_mask],
                         y_train,
                         validation_data=([X_test_input_ids, X_test_token_type_ids, X_test_attention_mask],
                         y_test),
                        epochs=5,
                        batch_size=8)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).